In [1]:
import requests

base_url = "http://35.200.185.69:8000/v2/autocomplete"
response = requests.get(f"{base_url}?query=aao")
print(response.json())

{'version': 'v2', 'count': 0, 'results': []}


In [2]:
response = requests.get(f"{base_url}?query=A")
print(response.headers)

{'date': 'Mon, 24 Mar 2025 01:01:39 GMT', 'server': 'uvicorn', 'content-length': '39', 'content-type': 'application/json'}


In [3]:
import requests
import time

BASE_URL = "http://35.200.185.69:8000/v2/autocomplete"
collected_names = set()

In [4]:
# Counters for tracking the number of successes and rate limit hits
successful_requests = 0
rate_limit_hits = 0

In [5]:
def fetch_results(query, max_retries=5):
    """ Fetch results from the API for a given query with rate limit handling. """
    global successful_requests, rate_limit_hits

    retries = 0
    backoff = 40  # Initial backoff time (seconds)

    while retries < max_retries:
        try:
            response = requests.get(f"{BASE_URL}?query={query}", timeout=5)

            # Handle successful response
            if response.status_code == 200:
                successful_requests += 1  # Increment success counter
                return response.json().get("results")

            # Handle rate limiting (HTTP 429)
            elif response.status_code == 429:
                rate_limit_hits += 1  # Increment rate limit hit counter
                print(f"Rate limit hit for query '{query}'. Retrying after {backoff}s...")
                time.sleep(backoff)
                backoff *= 2  # Exponential backoff (1s → 2s → 4s → ...)
                retries += 1

            # Handle other errors (e.g., 500, 403)
            else:
                print(f"Unexpected status {response.status_code} for query '{query}'.")
                break

        # Handle timeouts or connection errors
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            print(f"Request failed for query '{query}': {e}")
            time.sleep(backoff)
            backoff *= 2
            retries += 1

    print(f"Max retries reached for query '{query}'. Skipping...")
    return []

In [6]:
def common_char(arr):
  for i in range(len(arr[0])):
    for j in range(1, len(arr)):
      if i >= len(arr[j]) or arr[0][i] != arr[j][i]:
        return arr[0][:i]
  return arr[0]

In [7]:
alpha=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
num=['0','1','2','3','4','5','6','7','8','9']
special=[' ','+','-','.']

In [8]:
v1=alpha
v2=num+alpha
v3=special+num+alpha

In [9]:
v=v2
count=12

In [10]:
def next_query(query):
  if query[-1]!='z':
    idx=v.index(query[-1])
    query=query[:-1]+v[idx+1]
    return query
  else:
    while query[-1]=='z':
      if len(query)==1:
        return "-1"
      query=query[:-1]
    idx=v.index(query[-1])
    query=query[:-1]+v[idx+1]
    return query


In [11]:
query=v[0]
early_list=[]
api_request=0
while True:
    results = fetch_results(query)
    api_request+=1
    with open("output_v2.txt",'a') as f:
      for w in results:
        if w not in early_list:
          f.write(w+'\n')
      early_list=results
    if len(results) == count:
        query=common_char(results)
        query+=results[-1][len(query)]
    else:
        query=next_query(query)
        if query == "-1":
          break

print(f"API requests: {api_request}")

Rate limit hit for query '19y'. Retrying after 40s...
Rate limit hit for query '1azou'. Retrying after 40s...
Rate limit hit for query '22'. Retrying after 40s...
Rate limit hit for query '3abu5'. Retrying after 40s...
Rate limit hit for query '3ap'. Retrying after 40s...
Rate limit hit for query '48'. Retrying after 40s...
Rate limit hit for query '5i'. Retrying after 40s...
Rate limit hit for query '6j'. Retrying after 40s...
Rate limit hit for query '7o'. Retrying after 40s...
Rate limit hit for query '8s'. Retrying after 40s...
Rate limit hit for query 'a8'. Retrying after 40s...
Rate limit hit for query 'azd1c'. Retrying after 40s...
Rate limit hit for query 'azd9'. Retrying after 40s...
Rate limit hit for query 'b1'. Retrying after 40s...
Rate limit hit for query 'c5'. Retrying after 40s...
Rate limit hit for query 'desvu5'. Retrying after 40s...
Rate limit hit for query 'desvuu'. Retrying after 40s...
Rate limit hit for query 'dny'. Retrying after 40s...
Rate limit hit for query

In [12]:
# After calling the function, you can print the tracking statistics:
print(f"Successful requests: {successful_requests}")
print(f"Rate limit errors: {rate_limit_hits}")

Successful requests: 3108
Rate limit errors: 62
